# Spectral Inversion
Notebook illustrating the spectral inversion transform. Spectral inversion usual occurs during a frequency conversion stage where the frequency scheme mirrors the signal spectrum frequency response.

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from torchsig.signals.signal_types import Signal
import torchsig.transforms.functional as F
from torchsig.utils.defaults import default_dataset
from torchsig.utils.dsp import frequency_shift

Function for generating modulated test input signal.

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["qpsk"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

Create test data at some center frequency, then apply spectral inversion.

In [ ]:
N = 1024
qpsk_data_baseband = generate_qpsk_signal(num_iq_samples=N, scale=1.0).data
sample_rate = 1
cfo = sample_rate / 4
qpsk_data_if = frequency_shift(qpsk_data_baseband, cfo, sample_rate)

# spectral inversion
impaired_qpsk_data = F.spectral_inversion(data=qpsk_data_if)

Plot signal with and without spectral inversion - note the frequency mirror around the origin.

In [ ]:
freq_vec = np.arange(-1.0 / 2, 1.0 / 2, 1.0 / (N))
qpsk_fft = np.fft.fftshift(np.fft.fft(qpsk_data_if)) / np.sqrt(N)
impaired_qpsk_fft = np.fft.fftshift(np.fft.fft(impaired_qpsk_data)) / np.sqrt(N)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1)
ax.plot(freq_vec, 20 * np.log10(np.abs(qpsk_fft)), label="Input")
ax.plot(
    freq_vec,
    20 * np.log10(np.abs(impaired_qpsk_fft)),
    alpha=0.5,
    label="Spectrally Inverted",
)
ax.set_ylim([-70, 0])
ax.set_xlim([freq_vec[0], freq_vec[-1]])
ax.set_ylabel("Magnitude (log10)", fontsize="large")
ax.set_title("QPSK with/without Spectral Inversion")
ax.legend(fontsize="large", loc="upper left")
ax.grid()